In [1]:
import os
import flatiron.tf.models.unet as fimu

2025-02-19 02:33:37.120593: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739932417.140552  540959 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739932417.146421  540959 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-19 02:33:37.167985: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# CPU only
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [ ]:
uri = os.environ.get('SLACK_URI', '')
config = f'''
engine: tensorflow
model:
    input_width: 208
    input_height: 208
    input_channels: 3
    classes: 1
    filters: 16
    layers: 7
    # activation:
    # batch_norm:
    # output_activation:
    # kernel_initializer:
    # attention_gates:
    # attention_activation_1:
    # attention_activation_2:
    # attention_kernel_size:
    # attention_strides:
    # attention_padding:
    # attention_kernel_initializer:
dataset:
    source: /mnt/storage/projects/unet001/dset001/p-unet001_s-dset001_d-glom_v001
    ext_regex: npy
    labels: [3]
    label_axis: -1
    limit: 1000
    test_size: 0.2
    reshape: False
optimizer:
    name: sgd
compile:
    loss: jaccard_loss
    device: cpu
    metrics:
        - jaccard
        - dice
callbacks:
    project: unet001
    root: /mnt/storage/projects
train:
    batch_size: 16
logger:
    slack_url: 'https://hooks.slack.com/services/{uri}'
    slack_channel: dev
    timezone: 'America/Detroit'
'''
# pipe = fimu.UNetPipeline \
#     .from_string(config) \
#     .build() \
#     .compile() \
#     .train_test_split() \
#     .load() \
#     .train()

pipe = fimu.UNetPipeline \
    .from_string(config) \
    .run()

2025-02-19 02:46:04.069164: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739933164.088632  547202 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739933164.094324  547202 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-19 02:46:04.114905: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.

RUN TIME:
```0.01 seconds (0:00:00.007270)```
POST TIME:
```2025-02-18T21:46:06.731600-05:00```
CONFIG:
```compile:
    devi

May not total to 100% - Loading Dataset Files:   0%|          | 0/8 [00:00<?, ?it/s]

May not total to 100% - Loading Dataset Files:   0%|          | 0/2 [00:00<?, ?it/s]


RUN TIME:
```1.33 seconds (0:00:01.328370)```
POST TIME:
```2025-02-18T21:46:08.308460-05:00```
CONFIG:
```dataset:
    ext_regex: npy
    label_axis: -1
    labels:
    - 3
    limit: 1000
    reshape: false
    seed: null
    shuffle: true
    source: /mnt/storage/projects/unet001/dset001/p-unet001_s-dset001_d-glom_v001
    test_size: 0.2
```
   


Epoch 2/30


/home/ubuntu/pdm/envs/pdm-kVbOHlCT-dev-3.10/lib/python3.10/site-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['input']
Received: inputs=Tensor(shape=(None, 208, 208, 3))
  warnings.warn(msg)
I0000 00:00:1739933173.882310  547318 service.cc:148] XLA service 0x7f358c0171a0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1739933173.882356  547318 service.cc:156]   StreamExecutor device (0): Host, Default Version
2025-02-19 02:46:13.982660: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1739933176.656509  547318 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 1/63 ━━━━━━━━━━━━━━━━━━━━ 11:00 11s/step - dice: 0.0000e+00 - jaccard: 0.0000e+00 - loss: 0.4917